## Load the Motion Data

Data is in x and y coordinates for each pixel. Each sample will be an array of 10 (frames) x 40 x 40 (capture window) x 2 (x and y) dimensions. 

In [ ]:
import numpy as np
import os
import matplotlib
matplotlib.use('TkAgg') # For displaying animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from helper_functions import *
from normalize_data import *
from integral_try import *
from numpy import array
from sklearn import preprocessing as pp

In [ ]:
data_dir = 'sonic_pi_face/data/'

# Get list of data files
data_files = get_data_files(data_dir)

# Load data into a dictionary
# Note: Checks for incomplete data
data = get_gesture_data(data_files)

# Get number of `open-close` samples
len(data['open-close'])

## Normalize data sample and get integral image for feature detection

In [ ]:
# Load first data sample from `open-close`
sample = data['open-close'][0]

#normalize data for each frame and each axis(x,y)
sample = normalize_data(sample)
print('normalized_data.shape',sample.shape)
print('normalized_data',sample)
# calculate the integral image
iimage = i_image(sample[4])

# calculate the integral for the first quarter from the integral image
try_integral = get_integral(iimage,0,0,20,20)
print('Example integral of the 1st quarter of one frame: ' , try_integral)

## Visualize horizontal motion across frames

In [ ]:
sample = data['open-close'][5] # Fifth sample
frame = sample[5][:,:,0] # Middle frame
# Note: Second argument takes 0 for `x`, 1 for `y` coordinate
display_frames(sample,0) 